In [1]:
from nba_api.stats.static import players,teams
import pandas as pd
import requests
import sys
import os
import time
from datetime import datetime
start_year=2014
end_year=2026



def get_dates(start_year,end_year):
    dates=[]
    for year in range(start_year,end_year):
    
        for team in teams.get_teams():
            team_id=team['id']
            path = '../shot_data/team/'+str(year)+'/'+str(team_id)+'.csv'
            if os.path.exists(path):
                df=pd.read_csv(path)
    
                df=df[['PLAYER_ID','TEAM_ID','HTM','VTM','GAME_DATE','GAME_ID']]
                df.sort_values(by='GAME_DATE',inplace=True)
                df.drop_duplicates(inplace=True)
                df['year']=year
                dates.append(df)
    return pd.concat(dates)
dateframe=get_dates(start_year,end_year)

In [2]:
frames= []
count=0
index_master=pd.read_csv('game_report/index_master.csv')
index_master=index_master[index_master.team!='TOT']
index_master['team_id']=index_master['team_id'].astype(int)
index_master['nba_id']=index_master['nba_id'].astype(int)
master=[]
for year in range(start_year,end_year):
    # Load the game data for the specific year.
    df = pd.read_csv(f'game_report/year_files/{year}_games.csv')

    team_map=dict(zip(df['TEAM_ID'],df['TEAM_ABBREVIATION']))
    
    # Filter index_master for the current year.
    year_index = index_master[index_master['year'] == year].reset_index()

    # Process each unique date in the dataset.
    for date in df['date'].unique().tolist():
        datedf=df[df.date==date].reset_index(drop=True)
        datedf=datedf.drop_duplicates(subset=['PLAYER_ID','date'])
        # Filter game data by date.
        
        gameframe = dateframe[dateframe['GAME_DATE'] == date].reset_index()
        gameframe.rename(columns={'GAME_DATE':'date'},inplace=True)
        # Get the unique team and game data for the specific date from gameframe.
        
        to_merge = gameframe[['TEAM_ID', 'GAME_ID', 'date', 'year']].drop_duplicates().reset_index(drop=True)
        

        save_frame=datedf.merge(to_merge,on=['TEAM_ID','date','year'],how='left')

      
        save_frame.drop_duplicates(inplace=True)

        
        # Merge game data with index_master to ensure correct team alignment.
        # Match on 'player' and 'team' columns from index_master and 'TEAM_ID' from the game data.

        # Identify rows where the merge may have issues.
        if save_frame['GAME_ID'].isna().any():
        
            missing=save_frame[save_frame['GAME_ID'].isna()].reset_index(drop=True)

            save_frame.dropna(subset='GAME_ID',inplace=True)
            missing.drop(columns=['GAME_ID','TEAM_ID','TEAM_ABBREVIATION'],inplace=True)

            missing=missing.merge(gameframe,on=['PLAYER_ID','year','date'],how='left')
            missing['TEAM_ABBREVIATION']=missing['TEAM_ID'].map(team_map)
    
            save_frame=pd.concat([save_frame,missing])

        if save_frame['GAME_ID'].isna().any():
        
            missing=save_frame[save_frame['GAME_ID'].isna()].reset_index(drop=True)
            missing.drop(columns='GAME_ID',inplace=True)
            save_frame.dropna(subset='GAME_ID',inplace=True)
            missed=[]

            
            for missed_player in missing['PLAYER_ID'].unique().tolist():
                missing_frame=missing[missing.PLAYER_ID==missed_player].reset_index(drop=True)
       
                temp_index=year_index[year_index.nba_id==missed_player].reset_index(drop=True)
                team_id=temp_index.iloc[0]['team_id']
                team=temp_index.iloc[0]['team']
                missing_frame['TEAM_ID']=int(team_id)
                missing_frame['TEAM_ABBREVIATION']=team
                missing_frame= missing_frame.merge(to_merge,on=['TEAM_ID','date','year'],how='left')
                missed.append(missing_frame)

            missing=pd.concat(missed)
            save_frame=pd.concat([save_frame,missing])
        if save_frame['GAME_ID'].isna().any():
        
            missing=save_frame[save_frame['GAME_ID'].isna()].reset_index(drop=True)
            missing.drop(columns='GAME_ID',inplace=True)
            save_frame.dropna(subset='GAME_ID',inplace=True)
            missed=[]
            
            for missed_player in missing['PLAYER_ID'].unique().tolist():
                missing_frame=missing[missing.PLAYER_ID==missed_player].reset_index(drop=True)
                temp_index=year_index[year_index.nba_id==missed_player].reset_index(drop=True)
                team_id=temp_index.iloc[1]['team_id']
                team=temp_index.iloc[1]['team']
                missing_frame['TEAM_ID']=int(team_id)
                missing_frame['TEAM_ABBREVIATION']=team
                missing_frame= missing_frame.merge(to_merge,on=['TEAM_ID','date','year'],how='left')
                missed.append(missing_frame)

            missing=pd.concat(missed)
            save_frame=pd.concat([save_frame,missing])

        if save_frame['GAME_ID'].isna().any():
        
            missing=save_frame[save_frame['GAME_ID'].isna()].reset_index(drop=True)
            missing.drop(columns='GAME_ID',inplace=True)
            save_frame.dropna(subset='GAME_ID',inplace=True)
            missed=[]
            
            for missed_player in missing['PLAYER_ID'].unique().tolist():
                missing_frame=missing[missing.PLAYER_ID==missed_player].reset_index(drop=True)
                temp_index=year_index[year_index.nba_id==missed_player].reset_index(drop=True)
                team_id=temp_index.iloc[2]['team_id']
                team=temp_index.iloc[2]['team']
                missing_frame['TEAM_ID']=int(team_id)
                missing_frame['TEAM_ABBREVIATION']=team
                missing_frame= missing_frame.merge(to_merge,on=['TEAM_ID','date','year'],how='left')
                missed.append(missing_frame)

            missing=pd.concat(missed)
            save_frame=pd.concat([save_frame,missing])
            
        if save_frame['GAME_ID'].isna().any():
            missing=save_frame[save_frame['GAME_ID'].isna()]
            print('test point')
   
        
        # Remove any duplicate entries after the merge.
        save_frame.drop_duplicates(inplace=True)
        save_frame['GAME_ID']=save_frame['GAME_ID'].astype(int)
        master.append(save_frame)
    
        # Save each game by unique GAME_ID.
total=pd.concat(master)
total
            

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,CATCH_SHOOT_FG3M,CATCH_SHOOT_EFG_PCT,CATCH_SHOOT_FGM,year,date,playoffs,GAME_ID,index,HTM,VTM
0,201166,Aaron Brooks,Aaron,1.610613e+09,DEN,29.0,1,1,0,1.0,...,NaN,NaN,0,2014,20131030,False,21300016,NaN,NaN,NaN
1,1733,Al Harrington,Al,1.610613e+09,WAS,34.0,1,0,1,0.0,...,0.0,0.0,0,2014,20131030,False,21300007,NaN,NaN,NaN
2,201143,Al Horford,Al,1.610613e+09,ATL,28.0,1,0,1,0.0,...,NaN,1.0,3,2014,20131030,False,21300012,NaN,NaN,NaN
3,2744,Al Jefferson,Al,1.610613e+09,CHA,29.0,1,0,1,0.0,...,NaN,0.5,2,2014,20131030,False,21300009,NaN,NaN,NaN
4,202329,Al-Farouq Aminu,Al-Farouq,1.610613e+09,NOP,23.0,1,0,1,0.0,...,0.0,0.0,0,2014,20131030,False,21300011,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,1641705,Victor Wembanyama,Victor,1.610613e+09,SAS,20.0,1,1,0,1.0,...,1.0,0.3,1,2025,20241213,False,22401215,NaN,NaN,NaN
166,1631117,Walker Kessler,Walker,1.610613e+09,UTA,23.0,1,0,1,0.0,...,2.0,1.5,2,2025,20241213,False,22401214,NaN,NaN,NaN
167,1642530,Yuki Kawamura,Yuki,1.610613e+09,MEM,23.0,1,1,0,1.0,...,NaN,NaN,0,2025,20241213,False,22401211,NaN,NaN,NaN
168,203897,Zach LaVine,Zach,1.610613e+09,CHI,29.0,1,1,0,1.0,...,0.0,0.0,0,2025,20241213,False,22401212,NaN,NaN,NaN


In [3]:
for team in total['TEAM_ID'].unique():
    temp = total[total.TEAM_ID==team]
    temp = temp[temp.year==2014]
    print(temp.TEAM_ABBREVIATION.iloc[0])
    print(temp.POINTS.sum())

DEN
8572.0
WAS
8164.0
ATL
8159.0
CHA
7766.0
NOP
8159.0
BKN
8157.0
UTA
7791.0
PHX
8629.0
MIN
8777.0
CLE
8545.0
TOR
8279.0
DET
8286.0
GSW
8724.0
NYK
8117.0
IND
8624.0
ORL
7644.0
SAS
8573.0
BOS
7596.0
SAC
7813.0
MEM
8058.0
MIL
7599.0
OKC
8923.0
HOU
8780.0
MIA
8413.0
LAL
8428.0
POR
8753.0
PHI
6906.0
DAL
8597.0
LAC
9231.0
CHI
7584.0
